In [2]:
# cargar las librerías

import matplotlib.pyplot as plt
import h3
import pandas as pd 
import geopandas as gpd
import folium
import branca.colormap as cm
from shapely.geometry import Point

In [22]:
# cargar los geodataframe desde su directorio

barrios = gpd.read_file(r"C:\Users\UserMCR\Desktop\Agustín\censo2022\Proyecto\GeoPandasOptimizado\4326\barrios.shp")
radios = gpd.read_file(r"C:\Users\UserMCR\Desktop\Agustín\censo2022\Proyecto\GeoPandasOptimizado\4326\radios.shp")
union_2 = gpd.read_file(r"C:\Users\UserMCR\Desktop\Agustín\censo2022\Proyecto\proyecto_avanzado\union2\union2.shp")

In [4]:
# crear un dataframe con coordenadas

data = {"lat": [40.7128, 34.0522],
        "lng": [-74.0060, -118.2437]}
df = pd.DataFrame(data)

# convertir las coordenadas en un geodataframe
ejemplo = gpd.GeoDataFrame(df,
                       geometry = gpd.GeoSeries.from_xy(df.lng, df.lat))

# agregar columna con el índice H3
ejemplo["h3_index"] = ejemplo.apply(lambda row: h3.latlng_to_cell(row["lat"], row["lng"], 9), axis = 1)

print(ejemplo)

       lat       lng                     geometry         h3_index
0  40.7128  -74.0060   POINT (-74.00600 40.71280)  892a1072893ffff
1  34.0522 -118.2437  POINT (-118.24370 34.05220)  8929a1d7577ffff


#### Following "deep charts" video tutorial for h3 indexation

In [5]:
# set coordinates and resolution

lat = -45.8060000
lng = -67.4879869

In [6]:
# convert coordinates to H3

h3_index = h3.latlng_to_cell(lat, lng, 10) # resolution = 10
print(f"H3 Index: {h3_index}")

# esto me arroja el índice H3 de esas coordenadas

H3 Index: 8acf9e1531b7fff


In [7]:
# convert H3 to coordinates
# inverted process

lat_lng = h3.cell_to_latlng(h3_index)
print(f"Latitude/Longitude: {lat_lng}")

Latitude/Longitude: (-45.805744589836415, -67.48776340848417)


In [8]:
# get the geographic boundary of the H3 cell
# convert H3 to geometry boundary

boundary = h3.cell_to_boundary(h3_index)
print(f"Boundary: {boundary}")

Boundary: ((-45.80636110912891, -67.48741887243419), (-45.80585539213191, -67.48692168115137), (-45.80523887672223, -67.48726621775347), (-45.80512806873713, -67.48810793733912), (-45.805633780249515, -67.48860514090742), (-45.806250305231664, -67.48826061260495))


In [9]:
# map the boundary
## create a map

map = folium.Map(location = [lat, lng], zoom_start = 16)

# add the H3 cell to the map

folium.Polygon(locations = boundary, color = "blue", weight = 5).add_to(map)

# save the map

map

In [15]:
# import data

radios.head(1)

,area,perimetro,provincia,dpto,frac,radio,tipo,link,pob2022,viv2022,nombre,geometry
0,1983406.875,9684.93,26,021,07,01,U,260210701,329.0,123.0,None,"POLYGON ((2616543.465 4934097.157, 2616540.215..."


In [18]:
# create a fuction to parse each row to create H3 index string and geometry boundaries

def compute_h3_and_boundaries(row, resolution = 8):
    lat = row["geometry"]
    lng = row["geometry"]
    h3_index = h3.latlng_to_cell(lat, lng, resolution)
    boundaries = h3.cell_to_boundary(h3_index)
    return pd.Series([h3_index, boundaries])

In [19]:
# implement function and check the results

radios[["H3_Index", "Boundaries"]] = radios.apply(compute_h3_and_boundaries, axis = 1)

TypeError: must be real number, not Polygon

In [24]:
radios.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich